In [1]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from collections import defaultdict as dd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
import pandas as pd
import numpy as np

import pickle
import scipy

from scipy.sparse import save_npz, load_npz

In [2]:
data1 = []
with open('../data/raw/comp90051-2024s1-project-1/domain1_train_data.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

data2 = []
with open('../data/raw/comp90051-2024s1-project-1/domain2_train_data.json', 'r') as f:
    for line in f:
        data2.append(json.loads(line))

data_test = []
with open('../data/raw/comp90051-2024s1-project-1/test_data.json', 'r') as f:
    for line in f:
        data_test.append(json.loads(line))

In [4]:
data_1_additional = pd.read_csv('../data/curated/new_features/text_features/data1_additional.csv')
data_2_additional = pd.read_csv('../data/curated/new_features/text_features/data2_additional.csv')
data_test_additional = pd.read_csv('../data/curated/new_features/text_features/data_test_additional.csv')

domain_clf_future = pd.read_csv('../data/curated/new_features/domain_clf_future.csv')
domain_clf_future['pred_domain'] -= 1

In [6]:
data_1_additional['unique_words'] = data_1_additional['text'].apply(lambda x: len(set(x.strip('[]').split(','))))
data_2_additional['unique_words'] = data_2_additional['text'].apply(lambda x: len(set(x.strip('[]').split(','))))
data_test_additional['unique_words'] = data_test_additional['text'].apply(lambda x: len(set(x.strip('[]').split(','))))

In [7]:
data_1_additional['unique_word_ratio'] = data_1_additional['unique_words'] / data_1_additional['length']
data_2_additional['unique_word_ratio'] = data_2_additional['unique_words'] / data_2_additional['length']
data_test_additional['unique_word_ratio'] = data_test_additional['unique_words'] / data_test_additional['length']

In [8]:
for x in data1:
    x['perplexity'] = float(data_1_additional.loc[data_1_additional['id'] == x['id'], 'perplexity'].values[0])
    x['burstiness'] = float(data_1_additional.loc[data_1_additional['id'] == x['id'], 'burstiness'].values[0])
    x['length'] = float(data_1_additional.loc[data_1_additional['id'] == x['id'], 'length'].values[0])
    x['unique_word_ratio'] = float(data_1_additional.loc[data_1_additional['id'] == x['id'], 'unique_word_ratio'].values[0])

for x in data2:
    x['perplexity'] = float(data_2_additional.loc[data_2_additional['id'] == x['id'], 'perplexity'].values[0])
    x['burstiness'] = float(data_2_additional.loc[data_2_additional['id'] == x['id'], 'burstiness'].values[0])
    x['length'] = float(data_2_additional.loc[data_2_additional['id'] == x['id'], 'length'].values[0])
    x['unique_word_ratio'] = float(data_2_additional.loc[data_2_additional['id'] == x['id'], 'unique_word_ratio'].values[0])

for x in data_test:
    x['perplexity'] = float(data_test_additional.loc[data_test_additional['id'] == x['id'], 'perplexity'].values[0])
    x['burstiness'] = float(data_test_additional.loc[data_test_additional['id'] == x['id'], 'burstiness'].values[0])
    x['length'] = float(data_test_additional.loc[data_test_additional['id'] == x['id'], 'length'].values[0])
    x['unique_word_ratio'] = float(data_test_additional.loc[data_test_additional['id'] == x['id'], 'unique_word_ratio'].values[0])
    x['domain'] = int(domain_clf_future.loc[domain_clf_future['id'] == x['id'], 'pred_domain'].values[0])

In [9]:
# Save data from data1 to JSON file
with open('../data/curated/comp90051-2024s1-project-1/domain1_train_data.json', 'w') as f:
    for item in data1:
        json.dump(item, f)
        f.write('\n')

# Save data from data2 to JSON file
with open('../data/curated/comp90051-2024s1-project-1/domain2_train_data.json', 'w') as f:
    for item in data2:
        json.dump(item, f)
        f.write('\n')

# Save data from data_test to JSON file
with open('../data/curated/comp90051-2024s1-project-1/test_data.json', 'w') as f:
    for item in data_test:
        json.dump(item, f)
        f.write('\n')


In [10]:
data_1_additional.to_csv('../data/curated/comp90051-2024s1-project-1/data1_additional.csv', index=False)
data_2_additional.to_csv('../data/curated/comp90051-2024s1-project-1/data2_additional.csv', index=False)
data_test_additional.to_csv('../data/curated/comp90051-2024s1-project-1/data_test_additional.csv', index=False)

In [11]:
data1 = []
with open('../data/curated/comp90051-2024s1-project-1/domain1_train_data.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

data2 = []
with open('../data/curated/comp90051-2024s1-project-1/domain2_train_data.json', 'r') as f:
    for line in f:
        data2.append(json.loads(line))

data_test = []
with open('../data/curated/comp90051-2024s1-project-1/test_data.json', 'r') as f:
    for line in f:
        data_test.append(json.loads(line))